In [ ]:
%%writefile popup.html
<html>
<script>
function _click() {
    console.log( window.opener._widget.model);
    window.opener._widget.model.send('hello');
}
</script>
<body>
<input type="button" value="go" onclick="_click();" />
</body>
</html>

In [ ]:
from __future__ import print_function
from IPython.html import widgets
from IPython.display import display
from IPython.utils.traitlets import Unicode

In [ ]:
class TestWidget(widgets.DOMWidget):
    _view_name = Unicode('TestView', sync=True)
    
    def __init__(self, **kwargs):
        super(TestWidget, self).__init__(**kwargs)
        self.on_msg(self.print_msg)
    
    def print_msg(self, _, contents):
        print(contents)

In [ ]:
%%javascript

require(["widgets/js/widget", "widgets/js/manager"], 
        function(widget, manager){
    
    var TestView = IPython.DOMWidgetView.extend({
        render: function(){
            var that = this;
            window._widget = this;
            
            this.model.comm.on_msg($.proxy(this.on_msg, this));
            
            this.$button = $('<input />')
                .attr('type', 'button')
                .attr('value', 'go')
                .click(function() {
                    window._widget.model.send('hello');
                    that.send('hello from js');
                })
                .appendTo(this.$el);
            
            this.popup = window.open('popup.html', '', 
                                     'width=200,height=200,left=1700,top=100');
            
        },
        
        on_msg: function(msg) {
            var contents = msg.content.data;
            this.$el.append('<p>' + contents + '</p>');
        }
    });
    
    manager.WidgetManager.register_widget_view('TestView', TestView);
});

In [ ]:
my_widget = TestWidget()

In [ ]:
display(my_widget)